In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/qiita-love/data_analysis"

/content/drive/MyDrive/qiita-love/data_analysis


In [ ]:
#インストール
!pip install mecab-python3

#辞書インストール
!pip install unidic-lite

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pickle
from sklearn.metrics import mean_absolute_error

In [6]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [34]:
# 1月から12月までのデータを読み込んで結合しdfに格納
df = pd.DataFrame()
for i in range(1, 13):
    df_tmp = pd.read_csv(f"../data_collection/data/2020-{i:02}.csv", encoding="utf8")
    df = pd.concat([df, df_tmp], ignore_index=True)
df.head()

,likes_count,title,body,created_date,created_time,tags,followers_count,organization,items_count
0,1.0,品質担保に本気で取り組んでみている話,株式会社オズビジョンの @terra_yucco です。2020/01 現在、オズビジョン ...,2020-01-31,23:59:59,"['仕様', '品質管理', '品質', 'QCD']",43.0,OZvision Inc.,97.0
1,5.0,【Unity】Screen.safeAreaとiOSステータスバーの罠（？）,# Unityで取得できる Screen.safeAreaには罠（？）がある罠に遭遇したのは...,2020-01-31,23:56:36,"['iOS', 'Unity', 'SafeArea']",2.0,NaN,7.0
2,3.0,徹底攻略！“SATySFiのロゴ”の出し方,**SATySFiのロゴ**といえば、もちろんコレですね。![image-1a.png](h...,2020-01-31,23:54:15,['SATySFi'],157.0,NaN,65.0
3,0.0,【KPI】家系図テーブルの操作 1【oracle】,某炎の紋章の家系図見てて、TRPGなどに使えるかなと思って家系図テーブルの運用を考える。階層...,2020-01-31,23:50:44,['oracle'],1.0,NaN,10.0
4,0.0,プログラミング・フォロを組み立てる,#プログラミング・フォロmicro:bitを内蔵して６本足で自律して歩き回ることの出来る [...,2020-01-31,23:45:47,"['RaspberryPi', 'microbit']",3.0,NaN,11.0


In [35]:
# dfの各カラムのデータ型を確認
df.dtypes

likes_count        float64
title               object
body                object
created_date        object
created_time        object
tags                object
followers_count    float64
organization        object
items_count        float64
dtype: object

In [36]:
# dfの各カラムの欠損値の数を確認
df.isnull().sum()

likes_count             1
title                   0
body                    1
created_date            1
created_time            1
tags                    1
followers_count         2
organization       110525
items_count             2
dtype: int64

In [37]:
### 変更箇所 ###
# organizationに値が入っていればTrue、欠損していればFalseに置き換える、さらに数値に変換する
df["organization"] = df["organization"].notnull().astype(int)
df.head()

,likes_count,title,body,created_date,created_time,tags,followers_count,organization,items_count
0,1.0,品質担保に本気で取り組んでみている話,株式会社オズビジョンの @terra_yucco です。2020/01 現在、オズビジョン ...,2020-01-31,23:59:59,"['仕様', '品質管理', '品質', 'QCD']",43.0,1,97.0
1,5.0,【Unity】Screen.safeAreaとiOSステータスバーの罠（？）,# Unityで取得できる Screen.safeAreaには罠（？）がある罠に遭遇したのは...,2020-01-31,23:56:36,"['iOS', 'Unity', 'SafeArea']",2.0,0,7.0
2,3.0,徹底攻略！“SATySFiのロゴ”の出し方,**SATySFiのロゴ**といえば、もちろんコレですね。![image-1a.png](h...,2020-01-31,23:54:15,['SATySFi'],157.0,0,65.0
3,0.0,【KPI】家系図テーブルの操作 1【oracle】,某炎の紋章の家系図見てて、TRPGなどに使えるかなと思って家系図テーブルの運用を考える。階層...,2020-01-31,23:50:44,['oracle'],1.0,0,10.0
4,0.0,プログラミング・フォロを組み立てる,#プログラミング・フォロmicro:bitを内蔵して６本足で自律して歩き回ることの出来る [...,2020-01-31,23:45:47,"['RaspberryPi', 'microbit']",3.0,0,11.0


In [38]:
# dfの各カラムの欠損値の数を確認
df.isnull().sum()

likes_count        1
title              0
body               1
created_date       1
created_time       1
tags               1
followers_count    2
organization       0
items_count        2
dtype: int64

In [39]:
# 欠損値を含む行を削除
df = df.dropna()
# indexを振り直す
df = df.reset_index(drop=True)

In [40]:
# likes_countの値が1000以上の行を抽出
df[df["likes_count"] >= 1000].head()

,likes_count,title,body,created_date,created_time,tags,followers_count,organization,items_count
749,1386.0,Vue開発者のためのVue.jsベストプラクティス集15選,# はじめに**みなさん、Vue使ってますかー！**・・・・・・・（へんじがない。ただのしか...,2020-01-29,23:00:56,"['JavaScript', 'Vue.js', 'Nuxt']",131.0,0,15.0
1008,1523.0,Vue.jsで作成された、ちょっと面白くて役立ちそうなサービス,## [こちらに移行しました。(2020/05/16)](https://tech-blog...,2020-01-29,08:43:49,"['JavaScript', 'Bootstrap', 'ツール', 'Vue.js', '...",35.0,0,19.0
1459,1024.0,FFT（高速フーリエ変換）を完全に理解する話,"FFT(Fast Fourier Transform),高速フーリエ変換についての記事です。...",2020-01-27,22:00:06,"['アルゴリズム', 'math', 'AtCoder', '競技プログラミング']",163.0,0,28.0
3500,1062.0,2020年現在 Web系企業で採用されてる技術についてまとめてみた,# はじめに2020年も始まりましたね！タイトル通りですが、2020年現在スタートアップや大...,2020-01-22,10:43:23,"['初心者', 'ツール', 'まとめ', '初心者向け', 'プログラミング言語']",24.0,0,27.0
3868,4834.0,良いコードの書き方,# 概要チームによる継続的開発を前提としたコーディングのガイドライン。特定の言語を対象とした...,2020-01-21,10:50:39,"['Java', 'プログラミング', 'コーディング規約', 'チーム開発', 'Swift']",128.0,1,29.0


In [41]:
# likes_countの値が1000以上なら1000に置き換える
df.loc[df["likes_count"] >= 1000, "likes_count"] = 1000
df[df["likes_count"] >= 1000].head()

,likes_count,title,body,created_date,created_time,tags,followers_count,organization,items_count
749,1000.0,Vue開発者のためのVue.jsベストプラクティス集15選,# はじめに**みなさん、Vue使ってますかー！**・・・・・・・（へんじがない。ただのしか...,2020-01-29,23:00:56,"['JavaScript', 'Vue.js', 'Nuxt']",131.0,0,15.0
1008,1000.0,Vue.jsで作成された、ちょっと面白くて役立ちそうなサービス,## [こちらに移行しました。(2020/05/16)](https://tech-blog...,2020-01-29,08:43:49,"['JavaScript', 'Bootstrap', 'ツール', 'Vue.js', '...",35.0,0,19.0
1459,1000.0,FFT（高速フーリエ変換）を完全に理解する話,"FFT(Fast Fourier Transform),高速フーリエ変換についての記事です。...",2020-01-27,22:00:06,"['アルゴリズム', 'math', 'AtCoder', '競技プログラミング']",163.0,0,28.0
3500,1000.0,2020年現在 Web系企業で採用されてる技術についてまとめてみた,# はじめに2020年も始まりましたね！タイトル通りですが、2020年現在スタートアップや大...,2020-01-22,10:43:23,"['初心者', 'ツール', 'まとめ', '初心者向け', 'プログラミング言語']",24.0,0,27.0
3868,1000.0,良いコードの書き方,# 概要チームによる継続的開発を前提としたコーディングのガイドライン。特定の言語を対象とした...,2020-01-21,10:50:39,"['Java', 'プログラミング', 'コーディング規約', 'チーム開発', 'Swift']",128.0,1,29.0


In [42]:
# created_dateを今日から何日前に投稿されたかを表す数値に変換
df["created_date"] = pd.to_datetime(df["created_date"])
df["created_days_ago"] = (pd.to_datetime("today") - df["created_date"]).dt.days
df["created_month"] = df["created_date"].dt.month
df = df.drop("created_date", axis=1)
# created_timeを何時に投稿されたかに変換
df["created_time"] = df["created_time"].str[:2].astype(int)
df.head()

,likes_count,title,body,created_time,tags,followers_count,organization,items_count,created_days_ago,created_month
0,1.0,品質担保に本気で取り組んでみている話,株式会社オズビジョンの @terra_yucco です。2020/01 現在、オズビジョン ...,23,"['仕様', '品質管理', '品質', 'QCD']",43.0,1,97.0,1044,1
1,5.0,【Unity】Screen.safeAreaとiOSステータスバーの罠（？）,# Unityで取得できる Screen.safeAreaには罠（？）がある罠に遭遇したのは...,23,"['iOS', 'Unity', 'SafeArea']",2.0,0,7.0,1044,1
2,3.0,徹底攻略！“SATySFiのロゴ”の出し方,**SATySFiのロゴ**といえば、もちろんコレですね。![image-1a.png](h...,23,['SATySFi'],157.0,0,65.0,1044,1
3,0.0,【KPI】家系図テーブルの操作 1【oracle】,某炎の紋章の家系図見てて、TRPGなどに使えるかなと思って家系図テーブルの運用を考える。階層...,23,['oracle'],1.0,0,10.0,1044,1
4,0.0,プログラミング・フォロを組み立てる,#プログラミング・フォロmicro:bitを内蔵して６本足で自律して歩き回ることの出来る [...,23,"['RaspberryPi', 'microbit']",3.0,0,11.0,1044,1


In [43]:
# 区切る関数を定義
import MeCab

wakati = MeCab.Tagger("-Owakati")
def wakati_process(x):
    return ' '.join(wakati.parse(x).split())

In [46]:
# なぜか区切ることができない行が存在することが判明したため、その行を特定した
for i in range(len(df)):
    try:
        wakati_process(df['body'][i])
    except:
        print(i)

48169
51297


In [53]:
# なぜか区切ることができない行を削除する
df = df.drop([48169, 51297]).reset_index()

In [55]:
# body -> body_wakati, title -> title_wakati
df['body_wakati'] = df['body'].apply(wakati_process)
df['title_wakati'] = df['title'].apply(wakati_process)

In [66]:
# tagsを区切る関数を定義
def tags_process(x):
    x = eval(x)
    return ' '.join(x)

In [67]:
# tags -> tags_wakati
df['tags_wakati'] = df['tags'].apply(tags_process)

In [70]:
# 文の情報をTF-IDFでベクトル化してからSVDでn次元に削減する関数
def vectorize_text(text, n_components=15):
    tfidf = TfidfVectorizer()
    df_tfidf = tfidf.fit_transform(df[text])
    file = '../app/model_tfidf/trained_model_' + text + '2.pkl'
    pickle.dump(tfidf, open(file, 'wb'))

    svd = TruncatedSVD(n_components=n_components)
    df_svd = svd.fit_transform(df_tfidf)
    file = f'../app/model_svd/trained_model_' + text + '2.pkl'
    pickle.dump(svd, open(file, 'wb'))

    df_ret = pd.DataFrame(df_svd, columns=[f"{text}_{i}" for i in range(n_components)])
    return df_ret

In [ ]:
# 文の情報をTF-IDFでベクトル化してからSVDでn次元に削減する関数(2回目以降)
# def vectorize_text(text, n_components=15):
#     file = '../app/model_tfidf/trained_model_' + text + '2.pkl'
#     tfidf = pickle.load(open(file, 'rb'))
#     df_tfidf = tfidf.transform(df[text])
#     pickle.dump(tfidf, open(file, 'wb'))

#     file = f'../app/model_svd/trained_model_' + text + '2.pkl'
#     svd = pickle.load(open(file, 'rb'))
#     df_svd = svd.transform(df_tfidf)
#     pickle.dump(svd, open(file, 'wb'))

#     df_ret = pd.DataFrame(df_svd, columns=[f"{text}_{i}" for i in range(n_components)])
#     return df_ret

In [71]:
# 指定のカラムをベクトル化して結合(次元数を減らしました)
for col, n_components in [("title_wakati", 10), ("body_wakati", 15), ("tags_wakati", 10)]:
    df_vec = vectorize_text(col, n_components)
    # ベクトル化したカラムを結合
    df = pd.concat([df, df_vec], axis=1)
    # 欠損値を含む行を削除
    df = df.dropna()
    # 元のカラムを削除
    df = df.drop(col, axis=1)
df.head()

,index,likes_count,title,body,created_time,tags,followers_count,organization,items_count,created_days_ago,created_month,title_wakati_0,title_wakati_1,title_wakati_2,title_wakati_3,title_wakati_4,title_wakati_5,title_wakati_6,title_wakati_7,title_wakati_8,title_wakati_9,body_wakati_0,body_wakati_1,body_wakati_2,body_wakati_3,body_wakati_4,body_wakati_5,body_wakati_6,body_wakati_7,body_wakati_8,body_wakati_9,body_wakati_10,body_wakati_11,body_wakati_12,body_wakati_13,body_wakati_14,tags_wakati_0,tags_wakati_1,tags_wakati_2,tags_wakati_3,tags_wakati_4,tags_wakati_5,tags_wakati_6,tags_wakati_7,tags_wakati_8,tags_wakati_9
0,0,1.0,品質担保に本気で取り組んでみている話,株式会社オズビジョンの @terra_yucco です。2020/01 現在、オズビジョン ...,23,"['仕様', '品質管理', '品質', 'QCD']",43.0,1,97.0,1044,1,0.007736,-0.003193,0.002488,0.002550,-0.001070,0.000836,0.000781,-0.001298,-0.002406,-0.002972,0.238656,-0.112503,-0.093621,-0.173695,-0.012459,-0.044524,-0.011929,-0.051411,-0.015929,-0.011204,-0.012049,-0.008437,0.027635,0.006467,0.014977,1.368733e-05,2.008910e-08,0.000005,-4.095102e-07,-5.004381e-07,-7.352703e-06,-4.310959e-07,-7.373516e-06,0.000037,1.525091e-06
1,1,5.0,【Unity】Screen.safeAreaとiOSステータスバーの罠（？）,# Unityで取得できる Screen.safeAreaには罠（？）がある罠に遭遇したのは...,23,"['iOS', 'Unity', 'SafeArea']",2.0,0,7.0,1044,1,0.018596,-0.008382,0.002794,0.002646,0.000987,0.000767,0.007414,-0.010836,0.010072,-0.020980,0.109321,-0.016636,-0.009209,-0.023449,-0.002165,0.004874,0.005872,0.009207,0.027644,-0.007107,-0.014845,-0.016279,-0.014128,-0.021301,-0.022534,1.210768e-03,2.620920e-03,0.001778,7.853317e-03,-5.817146e-03,-3.055517e-03,4.685220e-03,3.927977e-02,0.018394,1.876220e-03
2,2,3.0,徹底攻略！“SATySFiのロゴ”の出し方,**SATySFiのロゴ**といえば、もちろんコレですね。![image-1a.png](h...,23,['SATySFi'],157.0,0,65.0,1044,1,0.002186,-0.000564,0.001560,0.000911,-0.000236,-0.000304,0.000700,-0.000033,-0.000662,-0.000934,0.125084,0.056831,-0.029882,0.015917,-0.016434,0.017988,0.016414,-0.023173,-0.001035,-0.006510,0.003896,0.015343,-0.020594,0.008265,-0.009513,8.616383e-07,2.390846e-06,0.000002,1.125516e-06,-1.477474e-06,2.620367e-07,-7.626441e-07,-2.761501e-07,0.000004,3.209422e-07
3,3,0.0,【KPI】家系図テーブルの操作 1【oracle】,某炎の紋章の家系図見てて、TRPGなどに使えるかなと思って家系図テーブルの運用を考える。階層...,23,['oracle'],1.0,0,10.0,1044,1,0.022961,-0.011467,0.004939,0.009141,-0.009195,0.006809,-0.001586,0.007251,-0.013172,-0.004998,0.072059,-0.000280,0.013129,0.074750,-0.009927,0.051719,-0.038828,0.023521,0.009797,0.013960,-0.019724,0.014486,-0.004777,0.038244,-0.010212,3.459229e-04,4.587952e-04,0.001353,2.844342e-03,-1.866346e-03,1.407584e-03,1.078255e-03,-2.669674e-04,0.000488,1.763603e-04
4,4,0.0,プログラミング・フォロを組み立てる,#プログラミング・フォロmicro:bitを内蔵して６本足で自律して歩き回ることの出来る [...,23,"['RaspberryPi', 'microbit']",3.0,0,11.0,1044,1,0.007950,0.001512,0.015459,0.023242,-0.011165,-0.007725,0.005255,-0.011692,-0.005090,-0.006878,0.141048,0.097299,0.000505,-0.011557,-0.001484,0.022416,0.001264,0.004000,0.026142,0.001294,0.001801,0.007319,-0.023670,-0.020373,0.013155,9.035879e-04,2.826039e-03,0.008442,4.066472e-03,-5.879046e-04,3.478473e-03,-1.607553e-03,1.728594e-05,0.002013,9.784016e-04


In [72]:
# body, title, tagsの削除
df = df.drop(['body', 'title', 'tags'], axis=1)

In [73]:
# trainとtestに分割する(likes_countを目的変数とする)
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=0)
train_y = train["likes_count"]
train_X = train.drop(["likes_count"], axis=1)
test_y = test["likes_count"]
test_X = test.drop(["likes_count"], axis=1)

In [74]:
# Randomforestでモデルの学習
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(train_X, train_y)

# テストデータの予測
y_pred = model.predict(test_X)

In [77]:
mean_absolute_error(test_y, y_pred)

10.657483664772727

In [ ]:
file = '../app/model/trained_rfr_model_2.pkl'
pickle.dump(model, open(file, 'wb'))

In [ ]:
model_rfr = pickle.load(open(file, 'rb'))
model_rfr.predict(test_X.iloc[[0]])

array([4.4])